In [263]:
import sympy
sympy.init_printing()

In [264]:
# DH 파라미터를 이용해 Homogeneous Transformation Matrix 만드는 함수
def DH(a, alpha, d, theta):
    T_rot_x = sympy.Matrix([[1,0,0,0],
                            [0,sympy.cos(alpha),-sympy.sin(alpha),0],
                            [0,sympy.sin(alpha),sympy.cos(alpha),0],
                            [0,0,0,1]]);
    T_trans_x = sympy.Matrix([[1,0,0,a],
                              [0,1,0,0],
                              [0,0,1,0],
                              [0,0,0,1]]);
    T_trans_z = sympy.Matrix([[1,0,0,0],
                              [0,1,0,0],
                              [0,0,1,d],
                              [0,0,0,1]]);
    T_rot_z = sympy.Matrix([[sympy.cos(theta),-sympy.sin(theta),0,0],
                            [sympy.sin(theta),sympy.cos(theta),0,0],
                            [0,0,1,0],
                            [0,0,0,1]]);
    T = T_rot_x*T_trans_x*T_trans_z*T_rot_z;
    return T

# 동차변환에서 회전행렬만 뽑아내는 함수
def get_R_from_T(T):
    R = T[0:3,0:3];
    return R

# 속도전파식을 이용하여 관절의 각속도를 구하는 함수
def get_angular_vel_R(T_i_ip1,w_i,thd_ip1):
    R_i_ip1 = get_R_from_T(T_i_ip1);
    R_ip1_i = sympy.transpose(R_i_ip1);
    Z_ip1_ip1 = sympy.Matrix([[0],[0],[1]]);
    w_ip1_ip1 = R_ip1_i@w_i + thd_ip1*Z_ip1_ip1;
    return w_ip1_ip1    

# 속도전파식을 이용하여 관절의 선속도를 구하는 함수
def get_linear_vel_R(T_i_ip1,w_i,v_i):
    R_i_ip1 = get_R_from_T(T_i_ip1);
    R_ip1_i = sympy.transpose(R_i_ip1);
    r_i_ip1 = T_i_ip1[0:3,3:4];
    v_ip1_ip1 = R_ip1_i@(v_i + w_i.cross(r_i_ip1));
    return v_ip1_ip1

# 선속도와 각속도를 입력으로 받아 자코비안을 구하는 함수
def get_Jacobian(v_0_n,w_0_n,q):
    J = sympy.zeros(6,len(q));
    i = 0;
    for q_i in q:
        j_i = [];
        j_i.append(sympy.simplify(sympy.diff(v_0_n,q_i)));
        j_i.append(sympy.simplify(sympy.diff(w_0_n,q_i)));
        J[:,i] = sympy.Matrix(j_i);
        i+=1;
    return J


#변환행렬
def rotX(alpha,a,b,c):
    T = sympy.Matrix([[1,0,0,a],
                      [0,sympy.cos(alpha),-sympy.sin(alpha),b],
                      [0,sympy.sin(alpha),sympy.cos(alpha),c],
                      [0,0,0,1]]);
    return T

def rotY(alpha,a,b,c):
    T = sympy.Matrix([[sympy.cos(alpha),0,sympy.sin(alpha),a],
                      [0,1,0,b],
                      [-sympy.sin(alpha),0,sympy.cos(alpha),c],
                      [0,0,0,1]]);
    return T

def rotZ(alpha,a,b,c):
    T = sympy.Matrix([[sympy.cos(alpha),-sympy.sin(alpha),0,a],
                      [sympy.sin(alpha),sympy.cos(alpha),0,b],
                      [0,0,1,c],
                      [0,0,0,1]]);
    return T


#Trans
def trans(a,b,c):
    T = sympy.Matrix([[1,0,0,a],
                      [0,1,0,b],
                      [0,0,1,c],
                      [0,0,0,1]]);
    return T

# HW2

좌표계 {A}가 z축으로 30도 회전,  
이때 얻어진 좌표계의 y축 으로 90도 회전,  
변환행렬은?

In [265]:
a = rotZ(sympy.rad(30),0,0,0)
b = rotY(sympy.rad(90),0,0,0)

In [266]:
a * b

⎡          √3    ⎤
⎢0   -1/2  ──   0⎥
⎢          2     ⎥
⎢                ⎥
⎢     √3         ⎥
⎢0    ──   1/2  0⎥
⎢     2          ⎥
⎢                ⎥
⎢-1   0     0   0⎥
⎢                ⎥
⎣0    0     0   1⎦

위 케이스가 모두 고정좌표계의 주축에 대해 발생한경우는?  
절대변환

In [267]:
b * a

⎡ 0     0   1  0⎤
⎢               ⎥
⎢      √3       ⎥
⎢1/2   ──   0  0⎥
⎢      2        ⎥
⎢               ⎥
⎢-√3            ⎥
⎢────  1/2  0  0⎥
⎢ 2             ⎥
⎢               ⎥
⎣ 0     0   0  1⎦

## 2-5
1. 오일러각을 구해라

In [268]:
a = trans(3,0,2)
b = rotY(sympy.rad(90),0,0,0)
c = rotX(sympy.rad(30),0,0,0)

In [269]:
a * b * c

⎡          √3    ⎤
⎢0   1/2   ──   3⎥
⎢          2     ⎥
⎢                ⎥
⎢    √3          ⎥
⎢0   ──   -1/2  0⎥
⎢    2           ⎥
⎢                ⎥
⎢-1   0    0    2⎥
⎢                ⎥
⎣0    0    0    1⎦

X의 경우 Z축 방향으로 -1 방향  
방향 코사인은 [0,0,-1]  
동차행렬과 일치

In [270]:
#Y의 경우 x는 -sin30, y는 cos30
th = sympy.rad(30)
sympy.Matrix([[sympy.sin(-th)],[sympy.cos(th)],[0]])

⎡-1/2⎤
⎢    ⎥
⎢ √3 ⎥
⎢ ── ⎥
⎢ 2  ⎥
⎢    ⎥
⎣ 0  ⎦

# HW3

RRR 형 동차변환행렬

In [271]:
l1,l2,l3,th1,th2,th3 = sympy.symbols('l1,l2,l3,th1,th2,th3')

In [272]:
T01 = DH(l1,0,0,th1)
T12 = DH(l2,sympy.rad(90),0,th2)
T23 = DH(l3,0,0,th3)
T34 = DH(0,0,0,0)

In [273]:
T04 = T01 * T12 * T23 * T34

In [274]:
sympy.simplify(T04)

⎡cos(th₁)⋅cos(th₂ + th₃)  -sin(th₂ + th₃)⋅cos(th₁)  sin(th₁)   l₁ + l₂⋅cos(th₁
⎢                                                                             
⎢sin(th₁)⋅cos(th₂ + th₃)  -sin(th₁)⋅sin(th₂ + th₃)  -cos(th₁)        (l₂ + l₃⋅
⎢                                                                             
⎢    sin(th₂ + th₃)            cos(th₂ + th₃)           0                    l
⎢                                                                             
⎣           0                        0                  0                     

) + l₃⋅cos(th₁)⋅cos(th₂)⎤
                        ⎥
cos(th₂))⋅sin(th₁)      ⎥
                        ⎥
₃⋅sin(th₂)              ⎥
                        ⎥
    1                   ⎦

## 2
RP형 동차변환행렬

In [275]:
l2,th,d = sympy.symbols('l2,th,d')
T01 = DH(0,0,0,sympy.rad(90)+th)
T12 = DH(0,sympy.rad(90),d,0)
T23 = DH(l2,0,0,0)

T03 = T01 * T12 * T23

In [276]:
T03

⎡-sin(th)  0  cos(th)  d⋅cos(th) - l₂⋅sin(th)⎤
⎢                                            ⎥
⎢cos(th)   0  sin(th)  d⋅sin(th) + l₂⋅cos(th)⎥
⎢                                            ⎥
⎢   0      1     0               0           ⎥
⎢                                            ⎥
⎣   0      0     0               1           ⎦

## 3
RR형 동차변환행렬

In [277]:
l2,l1, th1, th2 = sympy.symbols('l2,l1, th1, th2')

In [278]:
T01 = DH(0,0,l1,th1)
T12 = DH(0,-sympy.rad(90),0,th2)
T23 = DH(l2,sympy.rad(90),0,0)
T03 = T01 * T12 * T23

In [279]:
T03

⎡cos(th₁)⋅cos(th₂)  -sin(th₁)  sin(th₂)⋅cos(th₁)  l₂⋅cos(th₁)⋅cos(th₂)⎤
⎢                                                                     ⎥
⎢sin(th₁)⋅cos(th₂)  cos(th₁)   sin(th₁)⋅sin(th₂)  l₂⋅sin(th₁)⋅cos(th₂)⎥
⎢                                                                     ⎥
⎢    -sin(th₂)          0          cos(th₂)         l₁ - l₂⋅sin(th₂)  ⎥
⎢                                                                     ⎥
⎣        0              0              0                   1          ⎦

# 1
## 1 {A} 기준으로 Bp를 보면?

In [373]:
Rab = trans(3,5,9) * rotZ(sympy.rad(30),0,0,0)
bP = sympy.Matrix([[1],[3],[5],[1]])

In [374]:
Rab * bP

⎡ √3   3  ⎤
⎢ ── + ─  ⎥
⎢ 2    2  ⎥
⎢         ⎥
⎢3⋅√3   11⎥
⎢──── + ──⎥
⎢ 2     2 ⎥
⎢         ⎥
⎢   14    ⎥
⎢         ⎥
⎣    1    ⎦

## 2 역행렬을 구하시오

In [375]:
a = sympy.Matrix([[0.9077, 0.0506, 0.4166, 1],[0.2432, 0.7456, -0.6204, -2],[-0.3420, 0.6645, 0.6645, 3],[0,0,0,1]])

In [376]:
a.T

⎡0.9077  0.2432   -0.342  0⎤
⎢                          ⎥
⎢0.0506  0.7456   0.6645  0⎥
⎢                          ⎥
⎢0.4166  -0.6204  0.6645  0⎥
⎢                          ⎥
⎣  1       -2       3     1⎦

# Example chapter.2

In [377]:
# 2.1 rot(z,30) bP = [0,2,0] what is aP
rotZ(sympy.rad(30),0,0,0) * sympy.Matrix([[0],[2],[0],[1]])

⎡-1⎤
⎢  ⎥
⎢√3⎥
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣1 ⎦

In [378]:
#2.2 역행렬 구하세용

a = rotZ(sympy.rad(45), -7.07, 2.83, 0)

In [379]:
a.T

⎡ √2     √2       ⎤
⎢ ──     ──   0  0⎥
⎢ 2      2        ⎥
⎢                 ⎥
⎢-√2     √2       ⎥
⎢────    ──   0  0⎥
⎢ 2      2        ⎥
⎢                 ⎥
⎢  0     0    1  0⎥
⎢                 ⎥
⎣-7.07  2.83  0  1⎦

In [380]:
# 2.4 rot(z, 45) and trans(7,3,0) is {B}. when Bp is [5,3,0] what is Ap?
Rab = rotZ(sympy.rad(45),7,3,0)
Bp = sympy.Matrix([[5],[3],[0],[1]])

In [381]:
Rab * Bp

⎡ √2 + 7 ⎤
⎢        ⎥
⎢3 + 4⋅√2⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎣   1    ⎦

In [382]:
# 2.5 2.4의 동차행렬의 역변환
Rab.T

⎡ √2   √2      ⎤
⎢ ──   ──  0  0⎥
⎢ 2    2       ⎥
⎢              ⎥
⎢-√2   √2      ⎥
⎢────  ──  0  0⎥
⎢ 2    2       ⎥
⎢              ⎥
⎢ 0    0   1  0⎥
⎢              ⎥
⎣ 7    3   0  1⎦

In [383]:
#2.6  Find 절대변환과 상대변환
# 상대변환
trans(0,1,0) * rotZ(sympy.rad(30),0,0,0) * trans(1,1,0) * rotZ(sympy.rad(-30),0,0,0)

⎡           1   √3⎤
⎢1  0  0  - ─ + ──⎥
⎢           2   2 ⎥
⎢                 ⎥
⎢          √3   3 ⎥
⎢0  1  0   ── + ─ ⎥
⎢          2    2 ⎥
⎢                 ⎥
⎢0  0  1     0    ⎥
⎢                 ⎥
⎣0  0  0     1    ⎦

In [384]:
#절대변환
rotZ(sympy.rad(-30),0,0,0) * trans(1,1,0)* rotZ(sympy.rad(30),0,0,0) * trans(0,1,0)

⎡         1   √3⎤
⎢1  0  0  ─ + ──⎥
⎢         2   2 ⎥
⎢               ⎥
⎢         1   √3⎥
⎢0  1  0  ─ + ──⎥
⎢         2   2 ⎥
⎢               ⎥
⎢0  0  1    0   ⎥
⎢               ⎥
⎣0  0  0    1   ⎦

In [385]:
# 2.7
# rot(x, 90) tran(-3,0,0) rot(y, -90)
rotX(sympy.rad(90), 0 ,0 ,0)*trans(-3,0,0)*rotY(sympy.rad(-90), 0 ,0 ,0)

⎡0   0  -1  -3⎤
⎢             ⎥
⎢-1  0  0   0 ⎥
⎢             ⎥
⎢0   1  0   0 ⎥
⎢             ⎥
⎣0   0  0   1 ⎦

In [386]:
# 2.8
# rot(y, -90) trans(0,0,-3) rot(x,90)
rotY(sympy.rad(-90),0,0,0) * trans(0,0,-3) * rotX(sympy.rad(90),0,0,0)

⎡0  -1  0   3⎤
⎢            ⎥
⎢0  0   -1  0⎥
⎢            ⎥
⎢1  0   0   0⎥
⎢            ⎥
⎣0  0   0   1⎦

In [294]:
# Example 3
a3,a4,d1,d2,d5,d6,th1,th2,th3,th4,th5,th6 = sympy.symbols('a3,a4,d1,d2,d5,d6,th1,th2,th3,th4,th5,th6')
T01 = DH(0,0,d1,th1)
T12 = DH(0,sympy.rad(-90),d2,sympy.rad(-90) + th2)
T23 = DH(a3,0,0,th3)
T34 = DH(a4,0,0,sympy.rad(-90) + th4)
T45 = DH(0,sympy.rad(90),d5,th5)
T56 = DH(0,sympy.rad(-90),d6,th6)

In [295]:
sympy.simplify(T01 * T12 * T23 * T34 * T45 * T56)2 

⎡-(sin(th₁)⋅sin(th₅) + cos(th₁)⋅cos(th₅)⋅cos(th₂ + th₃ + th₄))⋅cos(th₆) + sin(
⎢                                                                             
⎢(-sin(th₁)⋅cos(th₅)⋅cos(th₂ + th₃ + th₄) + sin(th₅)⋅cos(th₁))⋅cos(th₆) + sin(
⎢                                                                             
⎢                    sin(th₆)⋅cos(th₂ + th₃ + th₄) + sin(th₂ + th₃ + th₄)⋅cos(
⎢                                                                             
⎣                                                       0                     

th₆)⋅sin(th₂ + th₃ + th₄)⋅cos(th₁)  (sin(th₁)⋅sin(th₅) + cos(th₁)⋅cos(th₅)⋅cos
                                                                              
th₁)⋅sin(th₆)⋅sin(th₂ + th₃ + th₄)  (sin(th₁)⋅cos(th₅)⋅cos(th₂ + th₃ + th₄) - 
                                                                              
th₅)⋅cos(th₆)                                          -sin(th₆)⋅sin(th₂ + th₃
                                                   

In [296]:
# Example 3
l1,l2,th1,th2,th3 = sympy.symbols('l1,l2,th1,th2,th3')
T01 = DH(0,0,0,th1)
T12 = DH(l1,0,0,th2)
T23 = DH(l2,0,0,th3)
Result = T01 * T12 * T23

In [297]:
sympy.simplify(Result)

⎡cos(th₁ + th₂ + th₃)  -sin(th₁ + th₂ + th₃)  0  l₁⋅cos(th₁) + l₂⋅cos(th₁ + th
⎢                                                                             
⎢sin(th₁ + th₂ + th₃)  cos(th₁ + th₂ + th₃)   0  l₁⋅sin(th₁) + l₂⋅sin(th₁ + th
⎢                                                                             
⎢         0                      0            1                 0             
⎢                                                                             
⎣         0                      0            0                 1             

₂)⎤
  ⎥
₂)⎥
  ⎥
  ⎥
  ⎥
  ⎦

In [302]:
a = 1
b = 2
c = sympy.atan2(a,b)

In [309]:
sympy.deg(c)

In [348]:
# 3.4
# euler Z-Y-Z , rot(z,th1) * rot(y, th2) * rot(z, th3)
th1,th2,th3 = sympy.symbols('th1,th2,th3')
result =rotZ(th1, 0,0,0) * rotY(th2, 0,0,0) * rotZ(th3, 0,0,0)

In [349]:
sympy.simplify(result)

⎡-sin(th₁)⋅sin(th₃) + cos(th₁)⋅cos(th₂)⋅cos(th₃)  -sin(th₁)⋅cos(th₃) - sin(th₃
⎢                                                                             
⎢sin(th₁)⋅cos(th₂)⋅cos(th₃) + sin(th₃)⋅cos(th₁)   -sin(th₁)⋅sin(th₃)⋅cos(th₂) 
⎢                                                                             
⎢              -sin(th₂)⋅cos(th₃)                                sin(th₂)⋅sin(
⎢                                                                             
⎣                       0                                                0    

)⋅cos(th₁)⋅cos(th₂)  sin(th₂)⋅cos(th₁)  0⎤
                                         ⎥
+ cos(th₁)⋅cos(th₃)  sin(th₁)⋅sin(th₂)  0⎥
                                         ⎥
th₃)                     cos(th₂)       0⎥
                                         ⎥
                             0          1⎦

In [350]:
x = sympy.sin(th2) * sympy.cos(th1)
y = sympy.sin(th1) * sympy.sin(th2)
z = sympy.cos(th2)

In [351]:
x

In [352]:
y

In [353]:
z

In [354]:
y/x # th1 = atan2(y/x)

In [355]:
(y/z)/sympy.sin(th1)

In [360]:
# Find Jacobian w00 = v00 = 0
th1,th2,l1,l2 = sympy.symbols('th1,th2,l1,l2')
th1d,th2d = sympy.symbols('th1d,th2d')
T01 = DH(0, 0, 0, th1)
T12 = DH(l1, 0, 0, th2)
T23 = DH(l2, 0, 0, 0)
T03 = T01 * T12 * T23

In [361]:
w00 = sympy.Matrix([[0],[0],[0]])
v00 = sympy.Matrix([[0],[0],[0]])

In [367]:
w11 = get_angular_vel_R(T01,w00,th1d)
v11 = get_linear_vel_R(T01,w00,v00)

w22 = get_angular_vel_R(T12,w11,th2d)
v22 = get_linear_vel_R(T12,w11,v11)

w33 = get_angular_vel_R(T23,w22,th2d)
v33 = get_linear_vel_R(T23,w22,v22)

q = sympy.Matrix([[th1d],[th2d]])

In [369]:
w03 = get_R_from_T(T03) * w33
v03 = get_R_from_T(T03) * v33

In [372]:
get_Jacobian(v03,w03,q)

⎡-l₁⋅sin(th₁) - l₂⋅sin(th₁ + th₂)  -l₂⋅sin(th₁ + th₂)⎤
⎢                                                    ⎥
⎢l₁⋅cos(th₁) + l₂⋅cos(th₁ + th₂)   l₂⋅cos(th₁ + th₂) ⎥
⎢                                                    ⎥
⎢               0                          0         ⎥
⎢                                                    ⎥
⎢               0                          0         ⎥
⎢                                                    ⎥
⎢               0                          0         ⎥
⎢                                                    ⎥
⎣               1                          2         ⎦